# Exploring Hacker News Posts

## 1 Introduction

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") receive votes and comments. Hacker News is extremely popular in technology and startup circles. 

The data is approximately 20000. The data can be downloaded at [Kaggle](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts).

The description for the data columns are as follows:

| **Column** | **Description** |
|:-----|:-----|
| **id** | unique identifier from Hacker News for the post |
| **title** | title of the post |
| **url** | URL that the posts links to, if the post has a URL |
| **num_points** | number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes |
| **num_comments** | number of comments on the post |
| **author** | username of the person who submitted the post |
| **created_at** | date and time of the post's submission |


The focus of this analysis is on the part of the data which has titles beginning with `Ask HN` and `Show HN`. This analysis seeks to answer the following the questions:
* Do `Ask HN` or `Show HN` receive more comments on average?
* Does the time of posts creation influence the number of comments on average?

## 2 Data Import

In [1]:
from csv import reader

# Read data and store as a list of lists
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

In [2]:
# Display the first five rows
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [3]:
# Extract the first row and remove it from the data
headers = hn[0]
hn = hn[1:]

print(headers, "\n")
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'] 

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## 3 Extracting the Ask HN and Show HN Posts

The data is then split into separate lists containing the `ask hn` and `show hn` posts respectively. The remaining posts are stored in the **other posts** list.

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Number of posts beginning with 'ask hn': ", len(ask_posts))
print("Number of posts beginning with 'show hn': ", len(show_posts))
print("Number of other posts: ", len(other_posts))

Number of posts beginning with 'ask hn':  1744
Number of posts beginning with 'show hn':  1162
Number of other posts:  17194


## 4 The Average Number of Comments for Ask HN and Show HN Posts 

In [5]:
# The average number of comments for Ask HN posts
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments / len(ask_posts)
print("The average number of comments for Ask HN posts are ", avg_ask_comments)


# The average number of comments for Show HN posts
total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments

avg_show_comments = total_show_comments / len(show_posts)
print("The average number of comments for Show HN posts are ", avg_show_comments)

The average number of comments for Ask HN posts are  14.038417431192661
The average number of comments for Show HN posts are  10.31669535283993


The outcome shows that the `Ask HN` posts receive more comments than the `Show HN` posts. 

## 5 Analyzing the `Ask HN` posts

Knowing that the `Ask HN` Posts receive more comments than the `Show HN` posts, the analysis is now going to focus on the `Ask HN` posts. 

The next step is to determine the time of the day when `Ask HN` posts are more likely to attract comments. To execute that,
* the number of `Ask HN` posts created at every hour of the day and the corresponding comments received needs to be calculated.
* the average number of comments `Ask HN` posts received by hour created should be computed.

### 5.1 The Number of Ask Posts and Comments Created by hour

In [6]:
import datetime as dt

In [7]:
# Creating a list of lists containing the date-time of the creation of 
# a post and the number of comments for the post
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

    
# Creating two dictionaries: the first one to store the number of posts
# per each hour of the day and the second one to store the number of 
# comments per each hour of the day
counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    date_time = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour = date_time.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

print(counts_by_hour, "\n")
print(comments_by_hour)

{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58} 

{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


### 5.2 The Average Number of Comments for `Ask HN` Posts by Hour

In [9]:
# Creating a list of lists to the store the average number of comments per 
# each hour of the day
avg_by_hour = []

for hr in comments_by_hour:
    # [the hour of the day, comments each hour/number of posts each hour]
    avg_by_hour.append([hr, comments_by_hour[hr]/counts_by_hour[hr]]) 
    
print(avg_by_hour)                      

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


### 5.3 Sorting and Printing Values in `avg_by_hour` list

The format above makes it difficult to identify the hours with the highest average number of comments. The next step is to sort the outcome in a format that is easily readable. 

In [10]:
# Creating a list of lists with swapped positions of the keys and values of
# the avg_by_hour list
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


In [11]:
# Sorting the swap_avg_by_hour list in descending order
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(row[1], "%H")
    hour_str = hour.strftime("%H:%M")
    avg_comment_hr = row[0]
    print(f"{hour_str}: {avg_comment_hr:.2f} average comments per post")

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


From the above results, the best hour to create a post to have a higher chance of receiving comments is 15:00 (Eastern Time in the US).

## 6 Conclusion

The aim of this analysis was the exploration of the Hacker News data to discover which of these post types (`Ask HN` or `Show HN`) attains more comments on the average and also find the best time for a news post to gets more comments.

At the end of the analysis, it was discovered that `Ask HN` posts receive more comments on the average (approximately 4 posts more) than the `Show HN` posts. It also showed that the best time to create a post in order to receive more comments is 15:00 (Eastern Time in the US).